Using only t2w sequence

In [15]:
from prostate158.transforms import get_base_transforms
from prostate158.utils import load_config
from prostate158.model import get_model
from monai.transforms import Compose
from monai.inferers import sliding_window_inference
import torch
from ipywidgets import interact
from matplotlib import pyplot as plt

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
config = load_config('tests/config/config.yaml') # change to 'tumor.yaml' for tumor segmentation
transforms = Compose(get_base_transforms(config=config))

In [18]:
model = get_model(config)
model.load_state_dict(torch.load('models_saved/anatomy_3_1_sequence.pt'), strict=False)
model.to(device=device)
model.eval();

In [19]:
images = transforms({'t2': './tests/input/027/t2.nii.gz', 't2_anatomy_reader1': './tests/input/027/t2_anatomy_reader1.nii.gz'})
image_t2w = images['t2'].to(device=device).unsqueeze(0)
input = torch.cat([image_t2w])

In [20]:
with torch.no_grad():
    roi_size = (128, 128, 128)
    sw_batch_size = 4
    outputs = sliding_window_inference(input, roi_size, sw_batch_size, model, overlap=0.5)

In [21]:
def plot(index):
    plt.figure("check", figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.title(f"image")
    plt.imshow(image_t2w[0, 0, :, :, index].cpu())
    plt.subplot(1, 3, 2)
    plt.title(f"label")
    plt.imshow(images["t2_anatomy_reader1"][0, :, :, index])
    plt.subplot(1, 3, 3)
    plt.title(f"output")
    plt.imshow(torch.argmax(outputs, dim=1).detach().cpu()[0, :, :, index])
    plt.show()

interact(plot, index=(0, images["t2_anatomy_reader1"].shape[-1] - 1, 1))

interactive(children=(IntSlider(value=69, description='index', max=138), Output()), _dom_classes=('widget-inte…

<function __main__.plot(index)>

In [26]:
def plot(index, channel):
    plt.figure("check", figsize=(18, 6))
    plt.subplot(1, 3, 1)
    plt.title(f"image")
    plt.imshow(image_t2w[0, 0, :, :, index].cpu())
    plt.subplot(1, 3, 2)
    plt.title(f"label")
    plt.imshow(images["t2_anatomy_reader1"][0, :, :, index])
    plt.subplot(1, 3, 3)
    plt.title(f"output")
    plt.imshow(outputs.detach().cpu()[0, channel, :, :, index], cmap='jet')
    plt.show()

interact(plot, index=(0, images["t2_anatomy_reader1"].shape[-1] - 1, 1), channel=(0, 2, 1))

interactive(children=(IntSlider(value=69, description='index', max=138), IntSlider(value=1, description='chann…

<function __main__.plot(index, channel)>